In [7]:
"""
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT

if you don't have pyaudio, then run

>>> pip install pyaudio

note: with 2048 samples per chunk, I'm getting 20FPS
when also running the spectrum, its about 15FPS
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [44]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK, exception_on_overflow = False)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_int)
    yfft = np.abs(yf[0:CHUNK])  / (128 * CHUNK)
    line_fft.set_ydata(yfft)
    
    # volume
    delta = np.max(data_np) - np.min(data_np)
    
    # max frequency
    xMinOffset = 1
    xMaxOffset = 2000
    yfft_trimmed = yfft[xMinOffset:xMaxOffset]
    maxIndex = list(yfft_trimmed).index(max(yfft_trimmed))
    
    print(max(yfft_trimmed), xf[maxIndex+xMinOffset], delta)
    
    plt.show()
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
88
0.22176835281083437 21.543722520762092
254
0.6788620373484391 64.63116756228628
255
0.6688808865253447 64.63116756228628
138
0.5714957279209263 64.63116756228628
255
0.37561028254143575 107.71861260381046
60
0.8292629637573361 43.087445041524184
47
0.6400679262868527 107.71861260381046
63
1.0137788739639184 43.087445041524184
57
1.117363674801249 64.63116756228628
51
0.8125050362159713 64.63116756228628
39
0.6394454694867471 107.71861260381046
51
0.8888485326555224 43.087445041524184
103
0.7616474665743839 64.63116756228628
119
0.6932603754050292 64.63116756228628
119
0.5433819250621275 64.63116756228628
89
0.46525876695056434 64.63116756228628
103
0.6342754331263853 43.087445041524184
45
0.7569707894149514 43.087445041524184
58
0.7789494445913846 64.63116756228628
46
1.0392328950581704 43.087445041524184
68
1.0155474589050058 64.63116756228628
57
0.859480694497217 43.087445041524184
97
1.0260289638200257 64.63116756228628
81
0.801069099556983 43.087445041524184
47
0.

48
0.5579453754768494 21.543722520762092
43
0.6306437327416524 43.087445041524184
45
0.6781262461206833 43.087445041524184
52
0.8916820333673605 64.63116756228628
254
0.4626947734919764 236.980947728383
84
0.8316088727945135 43.087445041524184
stream stopped
average frame rate = 3 FPS
